In [ ]:
import pulp
import pandas as pd
import numpy as np
from collections import Counter

def make_variables(df, name:str, criterion:str):
    return [ 
        {'var' : pulp.LpVariable(name + '_' + str(row_num), lowBound=0, upBound=1, cat='Integer'), \
         'cost' : row['cost'], \
         'xP' : row[criterion], \
         'team' : row['Team'], \
         'position' : row['Pos'], \
         'name' : row['Name']} \
         for row_num, row in df.iterrows() ]


def pick_team(xPdf, criterion : str, formation:tuple[int], total_cost:float):
    """Finds optimal FPL team from projected expected points.
    
    xPdf -- data frame with cost, xP, team, position, ID, and name information
    criterion -- columns which optimizer uses for xP players
    formation -- football formation used for starting 11
    total_cost -- maximum cost of squad
    """
    starter_formation_dict = {'G' : 1, 'D': formation[0], 'M' : formation[1], 'F' : formation[2]}
    sub_formation_dict = {'G' : 1, 'D': 5 - formation[0], 'M' : 5 - formation[1], 'F' : 3 - formation[2]}
    
    # LpProblem Set-Up
    score_modifiers = {'Starters' : 1, 'Bench' : 0, 'Captain' : 1} # How do selections contribute to GW score
    obj_func = 0 # Objective function to maximize score
    
    cost_func = 0
    
    squad_decisions = {'Starters' : [], 'Bench' : [], 'Captain' : []} # decision variables for LpProblem
    squad_totals = {'Starters' : 0, 'Bench' : 0, 'Captain' : 0}
    
    team_funcs = {}
    
    for team in xPdf['Team'].unique():
        team_funcs[team] = 0

    position_funcs = {'Starters' : {'G' : 0, 'D' : 0, 'M' : 0, 'F' : 0}, \
                      'Bench' : {'G' : 0, 'D' : 0, 'M' : 0, 'F' : 0}}
    
    for decision in squad_decisions:
        squad_decisions[decision] = make_variables(xPdf, decision, criterion)
        
        for variable_dict in squad_decisions[decision]:
            variable = variable_dict['var']
            xp = variable_dict['xP']
            cost = variable_dict['cost']
            
            obj_func += score_modifiers[decision] * variable * xp
            squad_totals[decision] += variable

            # COST TEAM AND POSITION CONSTRAINT FUNCTIONS
            if decision in ['Starters', 'Bench']:
                cost_func += variable * cost
                
                team = variable_dict['team']
                team_funcs[team] += variable
                
                position = variable_dict['position']
                position_funcs[decision][position] += variable

    problem = pulp.LpProblem('Optimal_Team', pulp.LpMaximize)
    # CONSTRAINTS
    problem += obj_func, 'Objective'
    problem += (cost_func <= total_cost), 'Cost'
    
    
    problem += (squad_totals['Starters'] == 11), 'Starters'
    problem += (squad_totals['Bench'] == 4), 'Bench'
    problem += (squad_totals['Captain'] == 0), 'Captain'
    for team in team_funcs:
        problem += (team_funcs[team] <= 3), team

    for category in position_funcs:
        for position in position_funcs[category]:
            if category == 'Starters':
                problem += (position_funcs['Starters'][position] == starter_formation_dict[position]), 'starter_' + position
            if category == 'Bench':
                problem += (position_funcs['Bench'][position] == sub_formation_dict[position]), 'bench_' + position

    problem.solve()

    return (squad_decisions, pulp.value(problem.objective))